In [2]:
import os

In [3]:
%pwd

'c:\\Users\\lpaes\\projects\\Xray_img_detection\\research'

In [4]:
#os.chdir('../')
%pwd

'c:\\Users\\lpaes\\projects\\Xray_img_detection'

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    validation_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
        config = self.config
        eval_config = EvaluationConfig(
            path_of_model = config.training.trained_model_path,
            training_data = config.data_ingestion.train_dir,
            validation_data = config.data_ingestion.val_dir,
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE,
                       )
        return eval_config

In [14]:
from urllib.parse import urlparse

class Evaluation:
    def __init__(self, config:EvaluationConfig):
        self.config = config 
    
    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(target_size = self.config.params_image_size[:-1],
                               batch_size = self.config.params_batch_size,
                               interpolation = 'bilinear',
                               class_mode='categorical',
                              ) 

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.validation_data,
            #subset='validation' # This one should turn on if the data origen for test and val are from the same folder
            shuffle=False,
            **dataflow_kwargs,
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(path=self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
    
    def save_score(self):
        scores = {'loss': self.score[0], 'accuracy':self.score[1]}
        save_json(path=Path('scores.json'), data=scores)

In [15]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e

[2023-07-27 15:35:59,708: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-27 15:35:59,714: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-27 15:35:59,715: INFO: common: created directory at: artifacts]
Found 4763 images belonging to 2 classes.
298/298 [==============================] - 874s 3s/step - loss: 0.9329 - accuracy: 0.8642
[2023-07-27 15:50:35,111: INFO: common: json file saved at: scores.json]
